In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
from tqdm import tqdm
import cv2
import time
import torchvision
import torch
import torchvision.transforms as transforms
import sys


import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from tqdm import tqdm_notebook, tnrange

import torch.utils.model_zoo as model_zoo
import torchvision.models as models
from torchvision import datasets
from tqdm.notebook import *
from torchcontrib.optim import SWA

import time
import matplotlib.pyplot as plt


transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                            std=[0.22803, 0.22145, 0.216989])
])
trans1 = transforms.ToPILImage()

from evaluation import *
from complementary import *
from Myutils import *
from training_engine import *
import optuna
import random

In [2]:
print(torch.version.__version__)

1.5.1


In [179]:
class PE_dataset_pipeline(torch.utils.data.Dataset):
    def __init__(self, Table, eval_tab, window = 48):
        self.slices = list(eval_tab['Exam'].unique())
        self.window = window
        self.Table = Table
        self.eval_tab = eval_tab
            
    def __getitem__(self, index):
        mini_batch = self.eval_tab[self.eval_tab.Exam == self.slices[index]].reset_index().drop(['index'], axis = 1)
        Study = mini_batch['Exam'][0]
        temp = self.Table[self.Table.StudyInstanceUID == Study].reset_index().drop(['index'], axis = 1)
        
        #input construction
        Input = torch.zeros(7*4)
        Softner = nn.Softmax(dim = 0)
        for i in range(len(mini_batch)):
            Input[i*7] = Softner(torch.Tensor([mini_batch['negative_detect'][i], mini_batch['positive_detect'][i]]))[1]
            Input[i*7 + 1] = mini_batch['right_detect'][i]
            Input[i*7 + 2] = mini_batch['right'][i]
            Input[i*7 + 3] = Softner(torch.Tensor([mini_batch['negative_left'][i], mini_batch['positive_left'][i]]))[1]
            Input[i*7 + 4] = Softner(torch.Tensor([mini_batch['negative_central'][i], mini_batch['positive_central'][i]]))[1]
            Input[i*7 + 5] = Softner(torch.Tensor([mini_batch['negative_chronic'][i], mini_batch['chronic'][i], mini_batch['acute'][i]]))[1]
            Input[i*7 + 6] = Softner(torch.Tensor([mini_batch['negative_chronic'][i], mini_batch['chronic'][i], mini_batch['acute'][i]]))[2]
        #output contruction
        Target = torch.zeros(9)
        Target[0] = temp['indeterminate'][0]
        Target[1] = temp['negative_exam_for_pe'][0]
        Target[2] = temp['central_pe'][0]
        Target[3] = temp['rightsided_pe'][0]
        Target[4] = temp['leftsided_pe'][0]
        Target[5] = temp['chronic_pe'][0]
        Target[6] = temp['acute_and_chronic_pe'][0]
        Target[7] = temp['rv_lv_ratio_gte_1'][0]
        Target[8] = temp['rv_lv_ratio_lt_1'][0]
        return([Input, Target])
    def __len__(self):
        return(len(self.slices))

In [180]:
class IntegNet(nn.Module):
    def __init__(self, trial):
        super(IntegNet, self).__init__()
        self.dropout_rate = trial.suggest_float("dropout_rate", 0, 1)
        fc2_input_dim = trial.suggest_int("fc2_input_dim", 500, 2000)
        fc3_input_dim = trial.suggest_int("fc3_input_dim", 400, 800)
        self.fc1 = nn.Linear(4*7, fc2_input_dim)
        self.fc2 = nn.Linear(fc2_input_dim, fc3_input_dim)
        self.fco = nn.Linear(fc3_input_dim, fc3_input_dim)
        self.fc3 = nn.Linear(fc3_input_dim, 9)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = F.relu(self.fco(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = self.fc3(x)
        return x.clamp(min = 0, max = 1)

In [181]:
def Exam_level_tab(tab):
    tij = pd.DataFrame(columns = tab.columns)
    Exams = tab['StudyInstanceUID'].unique()
    for i, exam in tqdm(enumerate(Exams)):
        mini_batch = tab[tab.StudyInstanceUID == exam].reset_index()
        tij = pd.concat([tij, mini_batch[:1]])
    return(tij)

In [182]:
tab = pd.read_csv('train_exams.csv')
eval_tab = pd.read_csv('eval_tab_filled.csv')

In [194]:
eval_tab[eval_tab.right > 0.5]

,Unnamed: 0,Unnamed: 0.1,Exam,slice,negative_detect,positive_detect,right_detect,right,negative_left,positive_left,negative_central,positive_central,negative_chronic,chronic,acute,qa_motion,qa_contrast,flow_artifact,true_filling_defect_not_pe
54,54,54,233cbc840568,ca3d2e4b34c8,-1.416342,1.375451,1.099154,0.635758,-0.653306,0.680855,-1.591668,1.515715,-0.659838,0.148167,0.222489,0,0,0,0
73,73,73,2352e610538e,21520f8a1a33,-1.260760,1.211276,0.988356,0.586111,-0.723150,0.738044,-1.568311,1.474504,-0.724205,0.186575,0.260107,0,0,0,0
133,133,133,240864001c6f,8d464097280c,-0.747034,0.742054,0.870357,0.519254,-0.145416,0.159226,-1.006315,0.932672,-0.199147,-0.040885,-0.084187,0,0,0,0
142,142,142,2427f80f06e9,5a5b54a9225c,-1.015329,0.978037,0.953707,0.570885,1.313378,-1.276227,0.609080,-0.620479,-0.290352,0.051563,-0.072806,0,0,0,0
146,146,146,242ef334aa07,28ffe231ac2f,1.488258,-1.452127,0.866939,0.527146,-0.697617,0.718299,-1.553887,1.459880,-0.619117,0.117034,0.198491,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25017,25017,25017,ff7234715ecb,1b9a1424581b,-1.439752,1.408643,0.979309,0.581309,-0.755050,0.780544,-1.568806,1.498395,-0.244001,-0.003852,-0.068085,0,0,0,0
25049,25049,25049,ff98e871bdea,9756adef4e2f,-1.442819,1.402807,0.967953,0.573174,-0.763475,0.790049,-1.608208,1.534832,-0.540888,0.109613,0.129289,0,0,0,0
25050,25050,25050,ff98e871bdea,4d04e28a2ee4,-1.056426,1.023519,1.030421,0.609573,-0.726522,0.743967,-1.566588,1.478117,-0.218703,0.006806,-0.097604,0,0,0,0
25089,25089,25089,ffcf51d622de,3f509daaef1e,2.120524,-2.062826,0.919218,0.546541,-0.620066,0.632757,-1.514167,1.431903,-0.529287,0.137780,0.091393,0,0,0,0


In [183]:
test_tab = pd.read_csv('test_exams.csv')
test_eval_tab = pd.read_csv('test_eval_tab.csv')

In [184]:
def evaluate(net, test_loader, cuda = True):
    Wbce = torch.Tensor([0.09202453988, 0.0736196319, 0.1877300613, 0.06257668712,
                         0.06257668712, 0.1042944785, 0.1042944785, 0.2346625767, 0.0782208589]).cuda()
    criterion1 = nn.BCELoss(Wbce, reduction = 'mean')
    net.eval()
    TP = 0
    hard_TP = 0
    FN = 0
    hard_FN = 0
    Positives = 0
    hard_Positives = 0
    Negatives = 0
    hard_Negatives = 0
    error = 0
    for i , (inputs, labels) in tqdm(enumerate(test_loader)):
        if cuda:
            net = net.cuda()
            inputs = inputs.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.FloatTensor)
        outputs = net(inputs)
        predicted = (outputs>0.5).float()
        TP += float(torch.logical_and((predicted[:, 1:] == labels[:, 1:].float()), (labels[:, 1:].float() == 1)).sum())
        hard_TP += float(torch.logical_and((predicted[:, 2:] == labels[:, 2:].float()), (labels[:, 2:].float() == 1)).sum())
        Positives += float((labels[:, 1:].float() == 1).sum())
        hard_Positives += float((labels[:, 2:].float() == 1).sum())
        FN += float(torch.logical_and((predicted[:, 1:] != labels[:, 1:].float()), (labels[:, 1:].float() == 0)).sum())
        hard_FN += float(torch.logical_and((predicted[:, 2:] != labels[:, 2:].float()), (labels[:, 2:].float() == 0)).sum())
        error = criterion1(outputs, labels).item()
    precision_positive = TP/Positives
    precision_negative = TP/(FN+TP+0.00001)
    hard_precision_positive = hard_TP/hard_Positives
    hard_precision_negative = hard_TP/(hard_TP + hard_FN + 0.00001)
    print(2*hard_precision_positive*hard_precision_negative/(hard_precision_positive+hard_precision_negative+0.000001))
    return(2*precision_positive*precision_negative/((precision_positive+precision_negative+0.000001)*np.exp(error)), error)

In [185]:
L = list(tab[tab.negative_exam_for_pe == 0]['StudyInstanceUID'].unique()) + list(tab[tab.negative_exam_for_pe == 1][:2000]['StudyInstanceUID'].unique())

In [188]:
def objective(trial):

    # Generate the model.
    net = IntegNet(trial)
    #net.load_state_dict(torch.load('best_Integnet.pth'))

    # Generate the optimizers.
    lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
    n_epoch = trial.suggest_discrete_uniform("epochs", 10, 100, 10)
    optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = n_epoch+10)

    train_dataset = PE_dataset_pipeline(tab[tab.StudyInstanceUID.isin(L)], eval_tab[eval_tab.Exam.isin(L)])
    test_dataset = PE_dataset_pipeline(test_tab, test_eval_tab)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 200, num_workers = 4)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1000)

    Wbce = torch.Tensor([0.09202453988, 0.0736196319, 0.1877300613, 0.06257668712,
                         0.06257668712, 0.1042944785, 0.1042944785, 0.2346625767, 0.0782208589]).cuda()
    criterion1 = nn.BCELoss(weight = Wbce)
    
    cuda = True
    train_acc_period = 5
                                                 
    loss_train = []
    loss_test = []
    total = 0
    net.train()
    if cuda:
        net = net.cuda()
    for epoch in tnrange(int(n_epoch)):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0
        if scheduler != None:
            print(scheduler.get_last_lr())
        for i, (inputs, labels) in tqdm(enumerate(train_loader)):
            # get the inputs
            if cuda:
                net = net.cuda()
                inputs = inputs.type(torch.cuda.FloatTensor)
                labels = labels.type(torch.cuda.LongTensor)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            st = time.time()
            outputs = net(inputs)
            loss = criterion1(outputs.float(), labels.float())
            loss.backward()
            optimizer.step()
            total += labels.size(0)
            # print statistics
            running_loss = 0.33*loss.item() + 0.66*running_loss
            
            predicted = (outputs>0.5).float()
            correct = float((predicted == labels.float()).sum())/(9*labels.size(0))
            #correct = float((pred == labels).sum())/labels.size(0)
            
            running_acc = 0.3*correct + 0.66*running_acc
            loss_train.append(running_loss)
            if i % train_acc_period == train_acc_period-1:
                print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
                print('[%d, %5d] acc: %.3f' %(epoch + 1, i + 1, running_acc))
                running_loss = 0.0
                total = 0
            inputs = None
            outputs = None
            labels = None
            predicted = None
        if scheduler != None:
            scheduler.step()
                                                 
        error = evaluate(net, test_loader)[0]

        trial.report(error, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
        print(error)
    
    accuracy = evaluate(net, test_loader)[0]
    return accuracy

In [190]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=90)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-10-21 20:43:00,340] A new study created in memory with name: no-name-39cd825b-db20-4d5c-b5e2-03d99fc0cf9f


[0.00025564514769325014]


[1,     5] loss: 0.714
[1,     5] acc: 0.625
[1,    10] loss: 0.724
[1,    10] acc: 0.724
[1,    15] loss: 0.866
[1,    15] acc: 0.723
[1,    20] loss: 1.936
[1,    20] acc: 0.598




0.0
0.0
[0.00025556728163491315]


[2,     5] loss: 0.445
[2,     5] acc: 0.635
[2,    10] loss: 0.464
[2,    10] acc: 0.712
[2,    15] loss: 0.541
[2,    15] acc: 0.712
[2,    20] loss: 1.486
[2,    20] acc: 0.667



...


0.4485709633770039


[2.9832237889693985e-05]


[1,     5] loss: 0.241
[1,     5] acc: 0.635
[1,    10] loss: 0.112
[1,    10] acc: 0.718
[1,    15] loss: 0.093
[1,    15] acc: 0.716
[1,    20] loss: 0.182
[1,    20] acc: 0.620



[I 2020-10-22 00:34:22,196] Trial 89 pruned. 



0.0

Study statistics: 
  Number of finished trials:  90
  Number of pruned trials:  69
  Number of complete trials:  21
Best trial:
  Value:  0.598976745051488
  Params: 
    dropout_rate: 0.664081226466108
    fc2_input_dim: 804
    fc3_input_dim: 670
    lr: 6.09968479503796e-05
    epochs: 30.0


In [177]:
import time
st = time.time()
tab.set_index('StudyInstanceUID').index.get_loc('fffda3f22362')
time.time() - st

0.0026597976684570312

In [178]:
import time
st = time.time()
tab[tab.StudyInstanceUID == 'fffda3f22362']
time.time() - st

0.0016422271728515625

In [191]:
def evaluate(net, test_loader, cuda = True):
    Wbce = torch.Tensor([0.09202453988, 0.0736196319, 0.1877300613, 0.06257668712,
                         0.06257668712, 0.1042944785, 0.1042944785, 0.2346625767, 0.0782208589]).cuda()
    criterion1 = nn.BCELoss(Wbce, reduction = 'mean')
    net.eval()
    TP = 0
    hard_TP = 0
    FN = 0
    hard_FN = 0
    Positives = 0
    hard_Positives = 0
    Negatives = 0
    hard_Negatives = 0
    error = 0
    for i , (inputs, labels) in tqdm(enumerate(test_loader)):
        if cuda:
            net = net.cuda()
            inputs = inputs.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.FloatTensor)
        outputs = net(inputs)
        predicted = (outputs>0.5).float()
        TP += float(torch.logical_and((predicted[:, 1:] == labels[:, 1:].float()), (labels[:, 1:].float() == 1)).sum())
        hard_TP += float(torch.logical_and((predicted[:, 2:] == labels[:, 2:].float()), (labels[:, 2:].float() == 1)).sum())
        Positives += float((labels[:, 1:].float() == 1).sum())
        hard_Positives += float((labels[:, 2:].float() == 1).sum())
        FN += float(torch.logical_and((predicted[:, 1:] != labels[:, 1:].float()), (labels[:, 1:].float() == 0)).sum())
        hard_FN += float(torch.logical_and((predicted[:, 2:] != labels[:, 2:].float()), (labels[:, 2:].float() == 0)).sum())
        error = criterion1(outputs, labels).item()
    precision_positive = TP/Positives
    precision_negative = TP/(FN+TP+0.00001)
    hard_precision_positive = hard_TP/hard_Positives
    hard_precision_negative = hard_TP/(hard_TP + hard_FN + 0.00001)
    print(2*hard_precision_positive*hard_precision_negative/(hard_precision_positive+hard_precision_negative+0.000001))
    return(2*precision_positive*precision_negative/((precision_positive+precision_negative+0.000001)*np.exp(error)), error)
class IntegNet(nn.Module):
    def __init__(self):
        super(IntegNet, self).__init__()
        self.dropout_rate = 0.664081226466108
        fc2_input_dim = 804
        fc3_input_dim = 670
        self.fc1 = nn.Linear(4*7, fc2_input_dim)
        self.fc2 = nn.Linear(fc2_input_dim, fc3_input_dim)
        self.fco = nn.Linear(fc3_input_dim, fc3_input_dim)
        self.fc3 = nn.Linear(fc3_input_dim, 9)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = F.relu(self.fco(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = self.fc3(x)
        return x.clamp(min = 0, max = 1)

# Generate the model.
net = IntegNet()

# Generate the optimizers.
lr = 6.09968479503796e-05
n_epoch = 200
optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = n_epoch+10)

L = list(tab[tab.negative_exam_for_pe == 0]['StudyInstanceUID'].unique()) + list(tab[tab.negative_exam_for_pe == 1][:2000]['StudyInstanceUID'].unique())
train_dataset = PE_dataset_pipeline(tab[tab.StudyInstanceUID.isin(L)], eval_tab[eval_tab.Exam.isin(L)])
L = list(test_tab[test_tab.negative_exam_for_pe == 0]['StudyInstanceUID'].unique()) + list(test_tab[test_tab.negative_exam_for_pe == 1][:2000]['StudyInstanceUID'].unique())
val_dataset = PE_dataset_pipeline(test_tab[test_tab.StudyInstanceUID.isin(L)], test_eval_tab[test_eval_tab.Exam.isin(L)])
test_dataset = PE_dataset_pipeline(test_tab, test_eval_tab)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 200, num_workers = 4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1000)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 1000)

Wbce = torch.Tensor([0.09202453988, 0.0736196319, 0.1877300613, 0.06257668712,
                     0.06257668712, 0.1042944785, 0.1042944785, 0.2346625767, 0.0782208589]).cuda()
criterion1 = nn.BCELoss(weight = Wbce)

cuda = True
train_acc_period = 5

loss_train = []
loss_test = []
total = 0
net.train()
best_acc = 0.5
if cuda:
    net = net.cuda()
for epoch in tnrange(int(n_epoch)):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    if scheduler != None:
        print(scheduler.get_last_lr())
    for i, (inputs, labels) in tqdm(enumerate(train_loader)):
        # get the inputs
        if cuda:
            net = net.cuda()
            inputs = inputs.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.LongTensor)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        st = time.time()
        outputs = net(inputs)
        #print(outputs[:,1:2].float(), labels[:,1:2].float())
        loss = criterion1(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        # print statistics
        running_loss = 0.33*loss.item() + 0.66*running_loss

        predicted = (outputs>0.5).float()
        correct = float((predicted == labels.float()).sum())/(9*labels.size(0))
        #correct = float((pred == labels).sum())/labels.size(0)

        running_acc = 0.3*correct + 0.66*running_acc
        loss_train.append(running_loss)
        if i % train_acc_period == train_acc_period-1:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
            print('[%d, %5d] acc: %.3f' %(epoch + 1, i + 1, running_acc))
            running_loss = 0.0
            total = 0
        inputs = None
        outputs = None
        labels = None
        predicted = None
    if scheduler != None:
        scheduler.step()
    
    acc = evaluate(net, test_loader)
    print("accuracy", acc[0])
    print("error", acc[1])
    
    if acc[0] > best_acc:
        best_acc = acc[0]
        torch.save(net.state_dict(), 'best_IntegNet.pth')
        print('saved')
        

accuracy = evaluate(net, test_loader)

[6.09968479503796e-05]


[1,     5] loss: 0.251
[1,     5] acc: 0.635
[1,    10] loss: 0.135
[1,    10] acc: 0.716
[1,    15] loss: 0.149
[1,    15] acc: 0.713
[1,    20] loss: 0.241
[1,    20] acc: 0.622




0.0
accuracy 0.0
error 0.06985291838645935
[6.099343523194726e-05]


[2,     5] loss: 0.064
[2,     5] acc: 0.635
[2,    10] loss: 0.072
[2,    10] acc: 0.718
[2,    15] loss: 0.072
[2,    15] acc: 0.714
[2,    20] loss: 0.182
[2,    20] acc: 0.639




0.3520136664303115
accuracy 0.20638633188729785
error 0.06898868829011917
[6.098319784040427e-05]


...

[199,     5] loss: 0.057
[199,     5] acc: 0.687
[199,    10] loss: 0.067
[199,    10] acc: 0.779
[199,    15] loss: 0.065
[199,    15] acc: 0.773
[199,    20] loss: 0.176
[199,    20] acc: 0.667




0.4426700776302751
accuracy 0.5585777046339122
error 0.10551810264587402
[4.120155941734482e-07]


[200,     5] loss: 0.057
[200,     5] acc: 0.687
[200,    10] loss: 0.067
[200,    10] acc: 0.779
[200,    15] loss: 0.065
[200,    15] acc: 0.773
[200,    20] loss: 0.176
[200,    20] acc: 0.667




0.44299155251403477
accuracy 0.5587833863175201
error 0.1055164486169815




0.44299155251403477


In [200]:
net.load_state_dict(torch.load('best_Integnet.pth'))
def evaluate(net, test_loader, cuda = True):
    Wbce = torch.Tensor([0.09202453988, 0.0736196319, 0.1877300613, 0.06257668712,
                         0.06257668712, 0.1042944785, 0.1042944785, 0.2346625767, 0.0782208589]).cuda()
    criterion1 = nn.BCELoss(Wbce)
    net.eval()
    correct = 0
    total = 0
    error = 0
    TP, FN, hard_TP, hard_FN = 0, 0, 0, 0
    Positives, hard_Positives = 0, 0
    for i , (inputs, labels) in tqdm(enumerate(test_loader)):
        if cuda:
            net = net.cuda()
            inputs = inputs.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.FloatTensor)
        outputs = net(inputs)
        predicted = (outputs>0.5).float()
        TP += float(torch.logical_and((predicted[:, 1:] == labels[:, 1:].float()), (labels[:, 1:].float() == 1)).sum())
        hard_TP += float(torch.logical_and((predicted[:, 4] == labels[:, 4].float()), (labels[:, 4].float() == 1)).sum())
        Positives += float((labels[:, 1:].float() == 1).sum())
        hard_Positives += float((labels[:, 4].float() == 1).sum())
        FN += float(torch.logical_and((predicted[:, 1:] != labels[:, 1:].float()), (labels[:, 1:].float() == 0)).sum())
        hard_FN += float(torch.logical_and((predicted[:, 4] != labels[:, 4].float()), (labels[:, 4].float() == 0)).sum())
        error = criterion1(outputs, labels).item()
        precision_positive = TP/Positives
        precision_negative = TP/(FN+TP+0.00001)
        hard_precision_positive = hard_TP/hard_Positives
        hard_precision_negative = hard_TP/(hard_TP + hard_FN + 0.00001)
        print(outputs, labels, total, correct, error)
        print(torch.logical_and((predicted[:, 1:] == labels[:, 1:].float()), (labels[:, 1:].float() == 1)))
        print(TP/Positives)
        print(TP, FN, Positives)
        print(hard_TP/hard_Positives)
        print(hard_TP, hard_FN, hard_Positives)
        #ok
    return(correct/9*total, error*labels.size(0)*9/total)
test_dataset = PE_dataset_pipeline(test_tab, test_eval_tab[100:])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1000)
accuracy = evaluate(net, test_loader)

tensor([[0.0000e+00, 4.5228e-01, 0.0000e+00, 4.0849e-01, 2.9576e-01, 9.2421e-02,
         0.0000e+00, 1.4409e-01, 3.5783e-01],
        [0.0000e+00, 1.7351e-01, 1.3660e-01, 7.1282e-01, 7.5644e-01, 0.0000e+00,
         0.0000e+00, 3.1926e-01, 4.8358e-01],
        [0.0000e+00, 4.9347e-01, 6.6002e-02, 3.5610e-01, 2.3603e-01, 1.1055e-01,
         0.0000e+00, 1.2459e-01, 3.0947e-01],
        [0.0000e+00, 7.4348e-01, 0.0000e+00, 1.8946e-01, 6.6141e-02, 0.0000e+00,
         0.0000e+00, 8.3502e-04, 2.0888e-01],
        [0.0000e+00, 5.2289e-01, 0.0000e+00, 3.1989e-01, 2.5062e-01, 8.5143e-02,
         0.0000e+00, 7.4703e-02, 3.2334e-01],
        [0.0000e+00, 6.8022e-01, 0.0000e+00, 2.3695e-01, 1.7431e-01, 4.3640e-02,
         0.0000e+00, 1.0433e-01, 2.0973e-01],
        [0.0000e+00, 5.1235e-01, 3.3400e-02, 4.2615e-01, 3.3003e-01, 6.7866e-02,
         0.0000e+00, 2.2800e-01, 3.1172e-01],
        [0.0000e+00, 7.7408e-01, 0.0000e+00, 1.0402e-01, 7.2332e-02, 4.4620e-02,
         0.0000e+00, 2.8767e-0

NameError: name 'ok' is not defined

In [197]:
eval_tab[eval_tab.positive_left > eval_tab.negative_left]

,Unnamed: 0,Unnamed: 0.1,Exam,slice,negative_detect,positive_detect,right_detect,right,negative_left,positive_left,negative_central,positive_central,negative_chronic,chronic,acute,qa_motion,qa_contrast,flow_artifact,true_filling_defect_not_pe
2,2,2,22c44fba113c,f2adb8925ab8,1.745316,-1.720462,0.607199,0.402854,-0.081019,0.096829,1.705787,-1.641119,0.854448,-0.626827,-0.641405,0,0,1,0
37,37,37,2310db914ba6,7d3492be95d1,2.412374,-2.352704,0.439322,0.323558,-0.560335,0.571616,1.278089,-1.248797,0.739591,-0.419069,-0.713284,0,0,0,0
54,54,54,233cbc840568,ca3d2e4b34c8,-1.416342,1.375451,1.099154,0.635758,-0.653306,0.680855,-1.591668,1.515715,-0.659838,0.148167,0.222489,0,0,0,0
61,61,61,2341d3b55832,e0f5190e3c60,1.235380,-1.200542,0.561965,0.383249,-0.078310,0.089450,1.832880,-1.753849,1.399536,-0.844866,-1.034250,0,0,0,0
70,70,70,234e30a9b491,e84608bca9df,2.351474,-2.294444,0.539553,0.368383,-0.378932,0.392606,2.081686,-1.987874,0.534398,-0.378379,-0.525779,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25065,25065,25065,ffadf3e7559f,fa82ae7f758a,2.104957,-2.051842,0.650419,0.419454,-0.235026,0.253867,0.544524,-0.545811,0.217567,-0.319932,-0.269226,0,0,0,0
25089,25089,25089,ffcf51d622de,3f509daaef1e,2.120524,-2.062826,0.919218,0.546541,-0.620066,0.632757,-1.514167,1.431903,-0.529287,0.137780,0.091393,0,0,0,0
25090,25090,25090,ffcf51d622de,b1d6b473d6ec,2.154882,-2.107390,0.788360,0.486851,-0.643877,0.651072,1.874208,-1.806931,1.048740,-0.629713,-0.831767,0,0,0,0
25099,25099,25099,ffe5280cd21d,8fe04e8291a7,1.715198,-1.665446,0.492986,0.341452,-0.472873,0.493456,2.062386,-1.968980,1.590873,-1.151602,-0.944298,0,0,0,0


In [198]:
eval_tab

,Unnamed: 0,Unnamed: 0.1,Exam,slice,negative_detect,positive_detect,right_detect,right,negative_left,positive_left,negative_central,positive_central,negative_chronic,chronic,acute,qa_motion,qa_contrast,flow_artifact,true_filling_defect_not_pe
0,0,0,22c44fba113c,2cb2c95c39c4,-1.256079,1.226184,0.612294,0.400113,0.992711,-0.965635,1.581507,-1.519417,0.568525,-0.462343,-0.502435,0,0,1,0
1,1,1,22c44fba113c,e0b9e88d36a6,-0.434502,0.427715,0.615801,0.409394,1.304161,-1.272452,0.399632,-0.422696,0.387828,-0.284932,-0.449033,0,0,0,0
2,2,2,22c44fba113c,f2adb8925ab8,1.745316,-1.720462,0.607199,0.402854,-0.081019,0.096829,1.705787,-1.641119,0.854448,-0.626827,-0.641405,0,0,1,0
3,3,3,22c44fba113c,138c91913b90,3.459264,-3.361123,-0.048203,0.092731,2.616429,-2.562468,2.743078,-2.666521,3.639309,-2.375117,-2.028206,0,0,1,0
4,4,4,22c5152fd61c,edb892fcb89a,3.078970,-2.992680,0.045447,0.133303,2.412557,-2.352313,2.380015,-2.292112,3.381478,-2.205732,-1.898424,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25111,25111,25111,fff9536823b1,1bbff36f49b6,2.863061,-2.787266,0.245312,0.216143,2.040442,-1.982904,2.305834,-2.213009,3.330920,-2.171576,-1.872502,0,0,0,0
25112,25112,25112,fffda3f22362,9d2faca4fef0,2.347847,-2.290215,0.521579,0.358635,0.912502,-0.885633,1.826843,-1.747761,1.106629,-0.765302,-0.798267,0,0,0,0
25113,25113,25113,fffda3f22362,d310c475d11a,1.988362,-1.940008,0.439846,0.318365,1.008849,-0.976478,2.015522,-1.928399,0.507286,-0.371346,-0.517759,0,0,0,0
25114,25114,25114,fffda3f22362,31f1f58ff99e,1.692219,-1.654929,0.431762,0.315558,0.937616,-0.909792,1.797570,-1.719360,0.304310,-0.391022,-0.284685,0,0,0,0


In [12]:
test_dataset = PE_dataset_pipeline(test_tab, test_eval_tab)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1000)
def algo_predictions(predictions, labels):
    Softner = nn.Softmax(dim = 0)
    Softner2 = nn.Softmax(dim = 0)
    labels = labels.cuda().float()
    Results = []
    Loss = 0
    total = 0
    Wbce = torch.Tensor([0.09202453988, 0.0736196319, 0.1877300613, 0.06257668712,
                         0.06257668712, 0.1042944785, 0.1042944785, 0.2346625767, 0.0782208589]).cuda()
    criterion1 = nn.BCELoss(Wbce, reduction = 'sum')
    for i, pred in tqdm(enumerate(predictions)):
        result = torch.zeros(9).cuda()
        Detections = []
        Right = []
        Left = []
        Central = []
        Chronic = []
        for j in range(4):
            Detections.append((Softner(pred[:2])[1].item() + pred[2] + 0.5)/2)
            Right.append(pred[3] + 0.5)
            Left.append(Softner(pred[4:6])[1].item())
            Central.append(Softner(pred[6:8])[1].item())
            Chronic.append((Softner(pred[8:11])[1], Softner2(pred[8:11])[2]))
        idx = 0
        Md = Detections[0]
        for j, d in enumerate(Detections):
            if d > Md:
                Md = d
                idx = j
        if Md > 0.5:
            result[1] = 1 - Detections[idx]
            result[2] = Central[idx]
            result[3] = Right[idx]
            result[4] = Left[idx]
            result[5] = Chronic[idx][0]
            result[6] = Chronic[idx][1]
        else:
            result[1] = 1
        Results.append(result)
        Loss += criterion1(result, labels[i])
        total += 1
        print(Loss)
    return(Loss/total)

In [13]:
algo_predictions(next(iter(test_loader))[0], next(iter(test_loader))[1])

tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(66.2883, device='cuda:0')
tensor(93.9877, device='cuda:0')
tensor(93.9877, device='cuda:0')
tensor(93.9877, device='cuda:0')
tensor(93.9877, device='cuda:0')
tensor(93.9877, device='cuda:0')
tensor(166.5338, device='cuda:0')
tensor(220.3068, device='cuda:0')
tensor(220.3068, device='cuda:0')
tensor(220.3068, device='cuda:0')
tensor(257.3926, device='cuda:0')
tensor(257.3926, device='cuda:0')
tensor(257.3926, device='cuda:0')
tensor(257.3926, device='cuda:0')
tensor(257.3926, device='cuda:0')
tensor(257.3926, device='cuda:0')
tensor(257.3926, device='cuda:0')
t

tensor(4533.2129, device='cuda:0')
tensor(4533.2129, device='cuda:0')
tensor(4533.2129, device='cuda:0')
tensor(4533.2129, device='cuda:0')
tensor(4576.5566, device='cuda:0')
tensor(4576.5566, device='cuda:0')
tensor(4576.5566, device='cuda:0')
tensor(4576.5566, device='cuda:0')
tensor(4576.5566, device='cuda:0')
tensor(4576.5566, device='cuda:0')
tensor(4623.0288, device='cuda:0')
tensor(4623.0288, device='cuda:0')
tensor(4623.0288, device='cuda:0')
tensor(4623.0288, device='cuda:0')
tensor(4660.1147, device='cuda:0')
tensor(4697.2007, device='cuda:0')
tensor(4697.2007, device='cuda:0')
tensor(4735.3296, device='cuda:0')
tensor(4773.4585, device='cuda:0')
tensor(4773.4585, device='cuda:0')
tensor(4773.4585, device='cuda:0')
tensor(4790.0229, device='cuda:0')
tensor(4790.0229, device='cuda:0')
tensor(4790.0229, device='cuda:0')
tensor(4833.3667, device='cuda:0')
tensor(4833.3667, device='cuda:0')
tensor(4833.3667, device='cuda:0')
tensor(4833.3667, device='cuda:0')
tensor(4833.3667, de

tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9887.9502, device='cuda:0')
tensor(9904.5146, device='cuda:0')
tensor(9904.5146, device='cuda:0')
tensor(9904.5146, device='cuda:0')
tensor(9904.5146, device='cuda:0')
tensor(9904.5146, device='cuda:0')
tensor(9966.6309, device='cuda:0')
tensor(9966.6309, device='cuda:0')
tensor(10009.9746, device='cuda:0')
tensor(10037.6738, device='cuda:0')
tensor(10037.6738, device='cuda:0')
tensor(10037.6738, device='cuda:0')
tensor(10037.6738, device='cuda:0')
tensor(10037.6738, device='cuda:0')
tensor(10074.7598, device='cuda:0')
tensor(10074.7598, device='cuda:0')
tensor(10074

RuntimeError: cuda runtime error (710) : device-side assert triggered at /pytorch/aten/src/THC/THCReduceAll.cuh:327